## Steel Defect Detection
This is starter kernel.<BR>
I used many kernels as a reference for creating this kernel. Especially,<BR>
["clear mask visualization and simple eda (GoldFish)"](https://www.kaggle.com/go1dfish/clear-mask-visualization-and-simple-eda) for image data visualization.<BR>
["RLE functions - Run Lenght Encode & Decode (Paulo Pinto)"](https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode) for rle encode-decode.<BR>
["Intro - chest xray, DICOM, viz, U-nets - full data (Jesper)"](https://www.kaggle.com/jesperdramsch/intro-chest-xray-dicom-viz-u-nets-full-data#Vanilla-Unet) for model

In [ ]:
#pip install git+git://github.com/fchollet/keras.git --upgrade --no-deps

In [ ]:
#pip install tensorflow-gpu==1.14

In [ ]:
import os
from pathlib import Path
from tqdm import tqdm
import json

import numpy as np # linear algebra
import pandas as pd
#pd.set_option("display.max_rows", 101)
import math

import cv2
import json
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.size"] = 10
import seaborn as sns
from PIL import Image

from collections import Counter
from collections import defaultdict

from keras.layers import *
from keras.models import Model
from keras.optimizers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
# import tensorflow as tf
# if tf.test.gpu_device_name():
#     print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
# else:
#     print("Please install GPU version of TF")
    

In [ ]:
#print(tf.__version__)

In [ ]:
DIRin1 = "../input/severstal-steel-defect-detection"
DIRin_train= "../input/prevtrain-steel"
print("DIRin1 =", os.listdir(DIRin1))
DIRtrain = os.path.join(DIRin1,"train_images")
DIRtest = os.path.join(DIRin1,"test_images")

DIRin2 = "../input/defect-detection-training"
#print("DIRin2 =", os.listdir(DIRin2))
# save path
#weights_path = os.path.join(DIRin2,"DefectDetection.h5")
#history_path = os.path.join(DIRin2,"DefectDetection_history.csv")

print("Num of Train img\t:",len(os.listdir(DIRtrain)))
print("Num of Test img\t\t:",len(os.listdir(DIRtest)))

Training is inadequate at the regular time of the one-hour competition. So I traind 50 epochs on my local PC (about 3 hours). And This kernel does not perform any further training, but uses pre-learned weights data.<BR>
If you want to train, set `NoTRAIN = False`.

In [ ]:
##### Training conditions ##### 
batch_size = 5

#NoTRAIN = True    # True:No further training, use pre-learned weights
#RESUME = False    # True:Resume Training, False:Start from the beginning

NoTRAIN = False    # True:No further training, use pre-learned weights
RESUME = False    # True:Resume Training, False:Start from the beginning

if RESUME:
    initial_epoch = 10    # initial_epoch when training resumes
else:
    initial_epoch = 0
epochs = initial_epoch + 10
steps_per_epoch = 200

## Image Data confirmation

In [ ]:
train_df = pd.read_csv(os.path.join(DIRin_train, "train.csv"))

train_df.head()

In [ ]:
# Transform class to column
train_df['fname'], train_df['cls'] = zip(*train_df['ImageId_ClassId'].str.split('_'))
train_df['cls'] = train_df['cls'].astype(int)
train_df = train_df.pivot(index='fname',columns='cls',values='EncodedPixels')
train_df['defects'] = train_df.count(axis=1)
#train_df.reset_index()
train_df.head(10)

In [ ]:
# Number of defects for each class
class_defects = len(train_df) - train_df.isnull().sum() # class毎の欠陥数
class_defects[:4]

In [ ]:
# check images size
train_size = defaultdict(int)
test_size = defaultdict(int)

for fPath in Path(DIRtrain).iterdir():
    img = Image.open(fPath)
    train_size[img.size] += 1
for fPath in Path(DIRtest).iterdir():
    img = Image.open(fPath)
    test_size[img.size] += 1
    
print("train_img_size :",train_size)
print("test_img_size  :",test_size)

## Visualization

In [ ]:
palet = [(250, 230, 20), (30, 200, 241), (200, 30, 250), (250,60,20)]

fig, ax = plt.subplots(1, 4, figsize=(6, 2))
for i in range(4):
    ax[i].axis('off')
    ax[i].imshow(np.ones((10, 40, 3), dtype=np.uint8) * palet[i])
    ax[i].set_title("class{}".format(i+1))

plt.show()

In [ ]:
def mask2rgba(mask):
    rgba_list = []
    for idx in range(4):    # idx: class id
        rgba = cv2.cvtColor(mask[:, :, idx], cv2.COLOR_GRAY2RGBA)
        rgba[:, :, :3] = rgba[:, :, :3] /255 * palet[idx]
        rgba_list.append(rgba)
    return rgba_list

In [ ]:
def make_mask(row_id):
    fname = train_df.iloc[row_id].name

    labels = train_df.iloc[row_id][:4]
    masks = np.zeros((256, 1600, 4), dtype=np.uint8)    # 4:class 1～4 (ch:0～3)

    for idx, label in enumerate(labels.values):
        if label is not np.nan:
            label = label.split(" ")
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            mask = np.zeros(256 * 1600, dtype=np.uint8)
            for pos, le in zip(positions, length):
                mask[pos:(pos + le)] = 255
            masks[:, :, idx] = mask.reshape(256, 1600, order='F')
    return fname, masks

In [ ]:
def show_mask_image(row_id, contour = True):
    name, mask = make_mask(row_id)
    img = cv2.imread(os.path.join(DIRtrain, name))

    if contour:
        for ch in range(4):
            contours, _ = cv2.findContours(mask[:, :, ch],
                            cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            for i in range(0, len(contours)):
                cv2.polylines(img, contours[i], True, palet[ch], 2)
    else:
        for ch in range(4):
            img[mask[:,:,ch]==255] = palet[ch]
        
    fig, ax = plt.subplots(figsize=(7,7))
    ax.set_title(name)
    ax.imshow(img)
    ax.axis('off')
    plt.show()

In [ ]:
# classify defects
idx_class_1 = list(filter(lambda r:not pd.isna(train_df.iloc[r,0]), range(len(train_df))))
idx_class_2 = list(filter(lambda r:not pd.isna(train_df.iloc[r,1]), range(len(train_df))))
idx_class_3 = list(filter(lambda r:not pd.isna(train_df.iloc[r,2]), range(len(train_df))))
idx_class_4 = list(filter(lambda r:not pd.isna(train_df.iloc[r,3]), range(len(train_df))))
# Nouber of defects class
idx_no_defect = list(filter(lambda r:train_df.iloc[r,4] == 0, range(len(train_df))))
idx_1_defect = list(filter(lambda r:train_df.iloc[r,4] == 1, range(len(train_df))))
idx_class_multi = list(filter(lambda r:train_df.iloc[r,4] >= 2, range(len(train_df))))

In [ ]:
# no defect sumple
for idx in idx_no_defect[:3]:
    show_mask_image(idx)

In [ ]:
# class_1 defect sumple (Yellow)
for idx in idx_class_1[:3]:
    show_mask_image(idx, contour=False)

In [ ]:
# class_2 defect sumple (lightblue)
for idx in idx_class_2[:3]:
    show_mask_image(idx, contour=True)

In [ ]:
# class_3 defect sumple (purple)
for idx in idx_class_3[:3]:
    show_mask_image(idx, contour=False)

In [ ]:
# class_4 defect sumple (red)
for idx in idx_class_4[:3]:
    show_mask_image(idx, contour=True)

In [ ]:
# contain multi class defects
for idx in idx_class_multi[:3]:
    show_mask_image(idx, contour=False)

## Model

In [ ]:
# # U-Net
# # https://www.kaggle.com/jesperdramsch/intro-chest-xray-dicom-viz-u-nets-full-data#Vanilla-Unet

# input_shape = (256, 1600, 1)
# inputs = Input(input_shape)

# c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (inputs)
# c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
# p1 = MaxPooling2D((2, 2)) (c1)

# c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
# c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
# p2 = MaxPooling2D((2, 2)) (c2)

# c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
# c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
# p3 = MaxPooling2D((2, 2)) (c3)

# c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
# c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
# p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

# c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (p4)
# c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (c5)
# p5 = MaxPooling2D(pool_size=(2, 2)) (c5)

# c55 = Conv2D(128, (3, 3), activation='relu', padding='same') (p5)
# c55 = Conv2D(128, (3, 3), activation='relu', padding='same') (c55)

# u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c55)
# u6 = concatenate([u6, c5])
# c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
# c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

# u71 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
# u71 = concatenate([u71, c4])
# c71 = Conv2D(32, (3, 3), activation='relu', padding='same') (u71)
# c61 = Conv2D(32, (3, 3), activation='relu', padding='same') (c71)

# u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c61)
# u7 = concatenate([u7, c3])
# c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
# c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

# u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
# u8 = concatenate([u8, c2])
# c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
# c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

# u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
# u9 = concatenate([u9, c1], axis=3)
# c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
# c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

# outputs = Conv2D(4, (1, 1), activation='sigmoid') (c9)

# model = Model(inputs=[inputs], outputs=[outputs])

# # Load pre-traind weights
# if (NoTRAIN or RESUME) and os.path.exists(weights_path):
#     model.load_weights(weights_path)

# model.summary()

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from keras import backend as K
from keras.layers import Layer


class MaxPoolingWithArgmax2D(Layer):
    def __init__(self, pool_size=(2, 2), strides=(2, 2), padding="same", **kwargs):
        super(MaxPoolingWithArgmax2D, self).__init__(**kwargs)
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides

    def call(self, inputs, **kwargs):
        padding = self.padding
        pool_size = self.pool_size
        strides = self.strides
        if K.backend() == "tensorflow":
            ksize = [1, pool_size[0], pool_size[1], 1]
            padding = padding.upper()
            strides = [1, strides[0], strides[1], 1]
            output, argmax = K.tf.nn.max_pool_with_argmax(
                inputs, ksize=ksize, strides=strides, padding=padding
            )
        else:
            errmsg = "{} backend is not supported for layer {}".format(
                K.backend(), type(self).__name__
            )
            raise NotImplementedError(errmsg)
        argmax = K.cast(argmax, K.floatx())
        return [output, argmax]

    def compute_output_shape(self, input_shape):
        ratio = (1, 2, 2, 1)
        output_shape = [
            dim // ratio[idx] if dim is not None else None
            for idx, dim in enumerate(input_shape)
        ]
        output_shape = tuple(output_shape)
        return [output_shape, output_shape]

    def compute_mask(self, inputs, mask=None):
        return 2 * [None]


class MaxUnpooling2D(Layer):
    def __init__(self, size=(2, 2), **kwargs):
        super(MaxUnpooling2D, self).__init__(**kwargs)
        self.size = size

    def call(self, inputs, output_shape=None):
        updates, mask = inputs[0], inputs[1]
        with K.tf.variable_scope(self.name):
            mask = K.cast(mask, "int32")
            input_shape = K.tf.shape(updates, out_type="int32")
            #  calculation new shape
            if output_shape is None:
                output_shape = (
                    input_shape[0],
                    input_shape[1] * self.size[0],
                    input_shape[2] * self.size[1],
                    input_shape[3],
                )
            self.output_shape1 = output_shape

            # calculation indices for batch, height, width and feature maps
            one_like_mask = K.ones_like(mask, dtype="int32")
            batch_shape = K.concatenate([[input_shape[0]], [1], [1], [1]], axis=0)
            batch_range = K.reshape(
                K.tf.range(output_shape[0], dtype="int32"), shape=batch_shape
            )
            b = one_like_mask * batch_range
            y = mask // (output_shape[2] * output_shape[3])
            x = (mask // output_shape[3]) % output_shape[2]
            feature_range = K.tf.range(output_shape[3], dtype="int32")
            f = one_like_mask * feature_range

            # transpose indices & reshape update values to one dimension
            updates_size = K.tf.size(updates)
            indices = K.transpose(K.reshape(K.stack([b, y, x, f]), [4, updates_size]))
            values = K.reshape(updates, [updates_size])
            ret = K.tf.scatter_nd(indices, values, output_shape)
            return ret

    def compute_output_shape(self, input_shape):
        mask_shape = input_shape[1]
        return (
            mask_shape[0],
            mask_shape[1] * self.size[0],
            mask_shape[2] * self.size[1],
            mask_shape[3],
        )

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) \
            / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
from keras.layers import Input
from keras.layers.convolutional import Convolution2D,Conv2D
from keras.layers.core import Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
#from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D


def segnet(input_shape, n_labels, kernel=2, pool_size=(2, 2), output_mode="softmax"):
    # encoder
    inputs = Input(shape=input_shape)

    conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Activation("relu")(conv_2)

    pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

    conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation("relu")(conv_3)
    conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = Activation("relu")(conv_4)

    pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

    conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation("relu")(conv_5)
    conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
    conv_6 = BatchNormalization()(conv_6)
    conv_6 = Activation("relu")(conv_6)
    conv_7 = Convolution2D(256, (kernel, kernel), padding="same")(conv_6)
    conv_7 = BatchNormalization()(conv_7)
    conv_7 = Activation("relu")(conv_7)

    pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

    conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
    conv_8 = BatchNormalization()(conv_8)
    conv_8 = Activation("relu")(conv_8)
    conv_9 = Convolution2D(512, (kernel, kernel), padding="same")(conv_8)
    conv_9 = BatchNormalization()(conv_9)
    conv_9 = Activation("relu")(conv_9)
    conv_10 = Convolution2D(512, (kernel, kernel), padding="same")(conv_9)
    conv_10 = BatchNormalization()(conv_10)
    conv_10 = Activation("relu")(conv_10)

    pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

    conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
    conv_11 = BatchNormalization()(conv_11)
    conv_11 = Activation("relu")(conv_11)
    conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(conv_11)
    conv_12 = BatchNormalization()(conv_12)
    conv_12 = Activation("relu")(conv_12)
    conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(conv_12)
    conv_13 = BatchNormalization()(conv_13)
    conv_13 = Activation("relu")(conv_13)

    pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
    print("Build enceder done..")

    # decoder

    unpool_1 = MaxUnpooling2D(pool_size)([pool_5, mask_5])

    conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_1)
    conv_14 = BatchNormalization()(conv_14)
    conv_14 = Activation("relu")(conv_14)
    conv_15 = Convolution2D(512, (kernel, kernel), padding="same")(conv_14)
    conv_15 = BatchNormalization()(conv_15)
    conv_15 = Activation("relu")(conv_15)
    conv_16 = Convolution2D(512, (kernel, kernel), padding="same")(conv_15)
    conv_16 = BatchNormalization()(conv_16)
    conv_16 = Activation("relu")(conv_16)

    unpool_2 = MaxUnpooling2D(pool_size)([conv_16, mask_4])

    conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_2)
    conv_17 = BatchNormalization()(conv_17)
    conv_17 = Activation("relu")(conv_17)
    conv_18 = Convolution2D(512, (kernel, kernel), padding="same")(conv_17)
    conv_18 = BatchNormalization()(conv_18)
    conv_18 = Activation("relu")(conv_18)
    conv_19 = Convolution2D(256, (kernel, kernel), padding="same")(conv_18)
    conv_19 = BatchNormalization()(conv_19)
    conv_19 = Activation("relu")(conv_19)

    unpool_3 = MaxUnpooling2D(pool_size)([conv_19, mask_3])

    conv_20 = Convolution2D(256, (kernel, kernel), padding="same")(unpool_3)
    conv_20 = BatchNormalization()(conv_20)
    conv_20 = Activation("relu")(conv_20)
    conv_21 = Convolution2D(256, (kernel, kernel), padding="same")(conv_20)
    conv_21 = BatchNormalization()(conv_21)
    conv_21 = Activation("relu")(conv_21)
    conv_22 = Convolution2D(128, (kernel, kernel), padding="same")(conv_21)
    conv_22 = BatchNormalization()(conv_22)
    conv_22 = Activation("relu")(conv_22)

    unpool_4 = MaxUnpooling2D(pool_size)([conv_22, mask_2])

    conv_23 = Convolution2D(128, (kernel, kernel), padding="same")(unpool_4)
    conv_23 = BatchNormalization()(conv_23)
    conv_23 = Activation("relu")(conv_23)
    conv_24 = Convolution2D(64, (kernel, kernel), padding="same")(conv_23)
    conv_24 = BatchNormalization()(conv_24)
    conv_24 = Activation("relu")(conv_24)

    unpool_5 = MaxUnpooling2D(pool_size)([conv_24, mask_1])

    conv_25 = Convolution2D(64, (kernel, kernel), padding="same")(unpool_5)
    conv_25 = BatchNormalization()(conv_25)
    conv_25 = Activation("relu")(conv_25)
    conv_26 = Convolution2D(32, (kernel, kernel), padding="same")(conv_25)
    conv_26 = BatchNormalization()(conv_26)
    conv_26 = Activation("relu")(conv_26)
    conv_26 = Convolution2D(16, (kernel, kernel), padding="same")(conv_26)
    conv_26 = BatchNormalization()(conv_26)
    conv_26 = Activation("relu")(conv_26)
    conv_26 = Convolution2D(8, (kernel, kernel), padding="same")(conv_26)
    conv_26 = BatchNormalization()(conv_26)
    conv_26 = Activation("relu")(conv_26)
    #conv_26 = Convolution2D(n_labels, (1, 1), padding="valid")(conv_26)
    
    outputs = Conv2D(4, (1, 1), activation='softmax') (conv_26)
#    conv_26 = BatchNormalization()(conv_26)
#     conv_26 = Reshape(
#         (input_shape[0] * input_shape[1], n_labels),
#         input_shape=(input_shape[0], input_shape[1], n_labels),
#     )(conv_26)

    #outputs = Activation(output_mode)(conv_26)
    print("Build decoder done..")

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])

    return model

In [ ]:
# def build_model(img_w, img_h, filters):
#     n_labels = 2

#     kernel = 3

#     encoding_layers = [
#         Conv2D(64, (kernel, kernel), input_shape=(img_h, img_w, 3), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(64, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         MaxPooling2D(),

#         Convolution2D(128, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(128, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         MaxPooling2D(),

#         Convolution2D(256, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(256, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(256, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         MaxPooling2D(),

#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         MaxPooling2D(),

#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         MaxPooling2D(),
#     ]

#     autoencoder =Sequential()
#     autoencoder.encoding_layers = encoding_layers

#     for l in autoencoder.encoding_layers:
#         autoencoder.add(l)

#     decoding_layers = [
#         UpSampling2D(),
#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),

#         UpSampling2D(),
#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(512, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(256, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),

#         UpSampling2D(),
#         Convolution2D(256, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(256, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(128, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),

#         UpSampling2D(),
#         Convolution2D(128, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(64, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),

#         UpSampling2D(),
#         Convolution2D(64, (kernel, kernel), padding='same'),
#         BatchNormalization(),
#         Activation('relu'),
#         Convolution2D(n_labels, (1, 1), padding='valid', activation="sigmoid"),
#         BatchNormalization(),
#     ]
#     autoencoder.decoding_layers = decoding_layers
#     for l in autoencoder.decoding_layers:
#         autoencoder.add(l)

#     autoencoder.add(Reshape((n_labels, img_h * img_w)))
#     autoencoder.add(Permute((2, 1)))
#     autoencoder.add(Activation('softmax'))

#     #with open('model_5l.json', 'w') as outfile:
#     #    outfile.write(json.dumps(json.loads(autoencoder.to_json()), indent=2))
#     autoencoder.summary()
#     return autoencoder

In [ ]:
#model=build_model(256,1600,10)

In [ ]:
# tem=input_shape[0] * input_shape[1]
# print(input_shape[0])

In [ ]:
input_shape = (256,1600,1)

model=segnet(input_shape,4)
model.summary()

In [ ]:
# !pip install q tensorflow==1.13.1
# !pip install q keras==1.2.2

In [ ]:
# !pip install q tensorflow==1.13.1
# !pip install q keras==1.2.2

In [ ]:
# def dice_coef(y_true, y_pred, smooth=1):
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) \
#             / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
# optimizer = Adam()
# model.compile(optimizer, 'binary_crossentropy', metrics=[dice_coef])

# Compile the model
#model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[dice_coef])

## Prepare Training

In [ ]:
import os
import json
import gc
import cv2
import keras
from keras import backend as K
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

def build_masks(rles, input_shape):
    depth = len(rles)
    masks = np.zeros((*input_shape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, input_shape)
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='../input/severstal-steel-defect-detection/train_images',
                 batch_size=32, dim=(256, 1600), n_channels=1,
                 n_classes=4, random_state=2019, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = f"{self.base_path}/{im_name}"
            img = self.__load_grayscale(img_path)
            
            # Store samples
            X[i,] = img

        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            image_df = self.target_df[self.target_df['ImageId'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            masks = build_masks(rles, input_shape=self.dim)
            
            y[i, ] = masks

        return y
    
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

In [ ]:
BATCH_SIZE = 2

train_idx, val_idx = train_test_split(
    non_missing_train_idx.index,  # NOTICE DIFFERENCE
    random_state=2019, 
    test_size=0.15
)

train_generator = DataGenerator(
    train_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)

val_generator = DataGenerator(
    val_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)

In [ ]:
# # Train Data Generator
# def Xy_generator(ids, batch_size):
#     Xs = []; ys = []
#     while True:
#         for i in ids:
#             name, mask = make_mask(i)
#             img = cv2.imread(os.path.join(DIRtrain, name),
#                              cv2.IMREAD_GRAYSCALE)
#             img = img[..., np.newaxis]    # Add channel axis
#             img = img / 255.           # 0～1
#             mask = mask / 255.         # 0～1
#             Xs.append(img); ys.append(mask)
#             if len(Xs) == batch_size:
#                 X = np.array(Xs); y = np.array(ys)
#                 Xs = []; ys = []
#                 yield [X, y]

In [ ]:
# # Train Data
# train_ids, val_ids = train_test_split(range(len(train_df)), test_size=0.2)
# train_gen = Xy_generator(train_ids, batch_size)
# val_gen = Xy_generator(val_ids, batch_size)

In [ ]:
# # generator test
# for X, y in Xy_generator(range(len(train_df)), 4):
#     break

# print('X.shape:',X.shape, '\ny.shape:',y.shape)

# row = 0
# # from train_df
# show_mask_image(row, contour=True)
# # from generator
# fig, axs = plt.subplots(5, figsize=(7,7))
# axs[0].imshow(X[row,:,:,0])
# axs[0].axis('off')
# axs[0].set_title(train_df.iloc[row].name)
# for i in range(4):
#     axs[i+1].imshow(y[row,:,:,i])
#     axs[i+1].axis('off')

## Training

In [ ]:
# Callback
#checkpoint = ModelCheckpoint("DefectDetection.h5", monitor='val_dice_coef',
                           #verbose=1,save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

In [ ]:
# # Fit
# # if NoTRAIN == False or RESUME:
# history = model.fit_generator(generator=train_gen,
#                               steps_per_epoch=10,
#                               #initial_epoch=initial_epoch,
#                               epochs=10,
#                               validation_data=val_gen,
#                               validation_steps = len(val_ids)//5,
#                               verbose=2,
#                              shuffle=True
#                               #callbacks=callbacks_list
#                                  )

In [ ]:
history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    use_multiprocessing=False,
    workers=1,
    epochs=5
)

In [ ]:
# Plot the loss and dice_coef curves
if (NoTRAIN or RESUME) and os.path.exists(history_path):
    hist_df = pd.read_csv(history_path)        # Load previous training history
if RESUME and os.path.exists(history_path):
    hist_df1 = pd.DataFrame(history.history)[['loss','val_loss','dice_coef','val_dice_coef']]
    hist_df = pd.concat([hist_df, hist_df1], ignore_index=True)    # Concat history
elif NoTRAIN == False and RESUME == False:
    hist_df = pd.DataFrame(history.history)[['loss','val_loss','dice_coef','val_dice_coef']]

# Plot
fig, ax = plt.subplots(1,2,figsize=(10, 3))

ax[0].plot(hist_df['loss'], color='b', label="Training loss")
ax[0].plot(hist_df['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist_df['dice_coef'], color='b', label="Training dice_coef")
ax[1].plot(hist_df['val_dice_coef'], color='r',label="Validation dice_coef")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
# Save history (for next Resume)
hist_df.to_csv("DefectDetection_history.csv", index=False)

## Predict

In [ ]:
# Load the weights that had the best score for predict
if NoTRAIN == False or RESUME:
    model.load_weights("DefectDetection.h5")

In [ ]:
# Binarize the mask output by NN
def binarize(masks, th = 0.5):
    # Maximum value of each channel per pixel
    mask_max = np.zeros_like(masks[:,:,0])
    mask_max = np.fmax(masks[:,:,0],masks[:,:,1])
    mask_max = np.fmax(mask_max,masks[:,:,2])
    mask_max = np.fmax(mask_max,masks[:,:,3])
    # Remove non-maximum pixels
    m = np.zeros_like(masks)
    for ch in range(4):
        m[:,:,ch] = (masks[:,:,ch] == mask_max) * masks[:,:,ch]
    # Binarization
    m = (m>th) * 1
    return m

In [ ]:
def show_predict_img(df, row):
    if df == "train_df":
        name = train_df.iloc[row].name
        img = cv2.imread(os.path.join(DIRtrain, name),
                             cv2.IMREAD_GRAYSCALE)
    else:
        if df == "submit_df":
            name = test_df.iloc[row//4,0].split('_')[0]
        elif df == "test_df":
            name = test_df.iloc[row,0]
        img = cv2.imread(os.path.join(DIRtest, name),
                             cv2.IMREAD_GRAYSCALE)

    img_ = img[..., np.newaxis]    # Add channel axis
    img_ = img_[np.newaxis, ...]    # Add batch axis
    img_ = img_ / 255.              # 0～1

    pred_masks = model.predict(img_)
    bin_masks = binarize(pred_masks[0, ...], 0.5)

    fig, axs = plt.subplots(5,2, figsize=(12, 6))
    axs[0,0].imshow(img)
    axs[0,0].axis('off')
    axs[0,0].set_title(name)
    axs[0,1].axis('off')
    axs[0,1].set_title("after binarize")
    for i in range(4):
        axs[i+1,0].imshow(pred_masks[0,:,:,i])
        axs[i+1,0].axis('off')
        axs[i+1,0].set_title('class '+ str(i+1))
        axs[i+1,1].imshow(bin_masks[:,:,i])
        axs[i+1,1].axis('off')
        axs[i+1,1].set_title('class '+ str(i+1))

In [ ]:
# predict sumple
show_predict_img("train_df", 4)

## Submission

In [ ]:
submit_df = pd.read_csv(os.path.join(DIRin1,'sample_submission.csv'))
submit_df['EncodedPixels'] = np.nan
submit_df.head(5)

In [ ]:
temp_df = pd.DataFrame()
temp_df['ImageId'] = submit_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_df = pd.DataFrame(temp_df['ImageId'].unique(), columns=['ImageId'])
test_df.head()

In [ ]:
def mask2rle(mask):
    pixels= mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[:-1:2]
    return ' '.join(str(x) for x in runs)

In [ ]:
def build_rles(masks):
    width, height, depth = masks.shape
    masks = binarize(masks, th = 0.5)
    rles = [mask2rle(masks[:, :, i]) for i in range(depth)]

    return rles

In [ ]:
# Predict test images
for i, line in tqdm(test_df.iterrows()):
    img = cv2.imread(os.path.join(DIRtest, line['ImageId']),
                     cv2.IMREAD_GRAYSCALE)
    img = img[..., np.newaxis]    # Add channel axis
    img = img[np.newaxis, ...]    # Add butch axis
    img = img / 255.              # 0～1
    pred_masks = model.predict(img)[0]
    rles = build_rles(pred_masks)
    for j in range(4):
        if len(rles[j])>0:
            submit_df.iloc[i*4+j,1] = rles[j]

In [ ]:
submit_df.head(30)

In [ ]:
# Number of Defect Detection
submit_df['EncodedPixels'].count()

In [ ]:
# Detected sumple
show_predict_img("submit_df",26)

In [ ]:
submit_df.to_csv('submission.csv', index=False)